**Exercise 1: A data scientist is running an AdaBoost classifier on a dataset with 100 observations. Answer the following:**

(a) (3 points) What is the weight initial weight of observation 72th in the training dataset? Be specific.

   The initial weight is 1/100. This is because initial weights are equal to 1/M, where M is the number of observations. All observations are weighted equally initially but are updated based on if they are misclassified or not. Misclassified observations will have a higher weight, so that it gets more attention while correct classifications recieve lower weights.

(b) (3 points) The 72th observation in the training dataset is misclassified by the first weak learner chosen by the data scientist. Is the new weight of the 72th observation in the training dataset (i.e., the weight assigned to the 72th observation after choosing the first weak classifier) larger or smaller than the weight assigned to that observation initially? Be specific.

Since it was misclassified, it will have a larger weight so it will get more attention in the sequence. If it were classified correctly, it would get a lower weight.

**Exercise 2: (4 points) Explain why AbaBoost.M1 is an ensemble learning algorithm? Be specific.**

Adaboost.M1 is an ensemble learning algorithm because it trains trees in a series. With adaboost.M1, a set of weak classifiers is connected in series such that each weak classifier tries to improve the classification of observations that were misclassified by the previous weak classifier. In doing so, boosting combines weak classifiers in series to create a strong classifier. Since it uses multiple trees, it is an ensemble learning algorithm.

**Exercise 3: (10 points) Suppose you are running AdaBoost.M1 (with η = 0.1) with 4 training examples. At the start of the current iteration, the four examples have the weights shown in the following table. Another column says if the weak classifier got them correct or incorrect. Determine the new
weights for these four examples, and fill in the corresponding entries in the table. Show all your work.**

refer to pdf submission (PNG has correct answer)

**Exercise 4: (4 points) If your AdaBoost ensemble under-fits the training dataset, what would you do to fix that? That is, which hyper-parameters should you tweak?**

You could increase the number of estimators (n_estimators) and increase the learning rate (learning_rate)

**Exercise 5: (4 points) For binary classification, which of the following statements are TRUE of AdaBoost with decision trees as learners?**
 - **(a) It usually has lower bias than a single decision tree.**
 - **(b) It is popular because it usually works well even before any hyper-parameter tuning.**
 - **(c) It assigns higher weights to observations that have been misclassified.**
 - **(d) It can train multiple decision trees in parallel.**
 - **(e) All of the above.**
 - **(f) None of the above.**

A, B, & C

Thought process: C is true and A is true. I've read adaboost is one of the best "out of the box" and "less susceptible to overfitting than other algorithms" which leads me to believe that B is true

**Exercise 6: (4 points) Which of the following is/are TRUE about gradient boosting trees?**
 - **(a) In gradient boosting trees, the decision trees are independent of each other.**
 - **(b) In gradient boosting trees, the decision trees are dependent of each other.**
 - **(c) It is a method for improving the performance by aggregating the results of several decision trees.**
 - **(d) (a) and (b)**
 - **(e) (a) and (c)**
 - **(f) (b) and (c)**
 - **(g) None of the above.**

(f) b and c

**Exercise 7: (6 points) In this course have covered two boosting frameworks. What is the main difference between AdaBoost and Gradient Boosting? Be specific.**

Gradient boosting trains the learners and reduces the weak learners' loss functions based on the model's residuals and Adaboost focuses on training based on prior misclassified observations by weighting them higher. 

**Exercise 8: Consider the framingham.csv data file. The dataset is publically available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD). The dataset provides the patients? information. It includes over 4,000 records and 15 attributes. Each attribute is a potential risk factor. There are both demographic, behavioral and medical risk factors. In Python, answer the following:**

(a) (5 points) Using the pandas library, read the csv data file and create a data-frame called
heart. Remove the observations with missing values.

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the datafile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
#dropping observations w/ missing values
heart=heart.dropna()

(b) (45 points) Based on the different interactions that we have so far with this data file, it seems that age, totChol, sysBP, BMI, heartRate, and glucose are the most important predictor variables when it comes to predict the likelihood of TenYearCHD. Then do the following:
- (i) Using TenYearCHD as the target variable and the other variables as the input variables, split the data into two data-frames (taking into account the proportion of 0s and 1s): train (80%) and test (20%).
- (ii) Using the train data-frame, build an random forest model (with 500 trees and max tree depth equal 3) in which: age, totChol, sysBP,BMI, heartRate, and glucose are the predictor variables, and TenYearCHD is the target variable. Using this model, predict the risk of coronary disease of the patients in the test data-frame. Using 10% as cutoff value, report the accuracy and recall.
- (iii) Using the train data-frame, build an AdaBoost model (with 500 trees, max tree depth equal 3, and learning rate equal to 0.01) in which: age, totChol, sysBP,BMI, heartRate, and glucose are the predictor variables, and TenYearCHD is the target variable. Using this model, predict the risk of coronary disease of the patients in the test data-frame. Using 10% as cutoff value, report the accuracy and recall.
- (iv) Using the train data-frame, build an gradient boosting model (with 500 trees, max tree depth equal 3, and learning rate equal to 0.01) in which: age, totChol, sysBP,BMI, heartRate, and glucose are the predictor variables, and TenYearCHD is the target variable. Using this model, predict the risk of coronary disease of the patients in the test data-frame. Using 10% as cutoff value, report the accuracy and recall.

Repeat (i)-(iv) 100 times and compute the average accuracy and average recall for each of the model. What model would you use to predict TenYearCHD based on the accuracy and recall?

In [3]:
# defining the input and target variables
x = heart[['age','totChol','sysBP','BMI','heartRate','glucose']]
y = heart['TenYearCHD']

rf_recall = list()
ada_recall = list()
gb_recall = list()

rf_accuracy = list()
ada_accuracy = list()
gb_accuracy = list()

for i in range (0,100):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)
    
    #random forest model
    rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(x_train, y_train)
    #predict on test
    rf_pred = rf_md.predict_proba(x_test)[:,1]
    #changing likelihoods to labels w/ 10% cutoff
    rf_labels = np.where(rf_pred < .1, 0, 1)
    rf_recall.append(recall_score(y_test, rf_labels))
    rf_accuracy.append(accuracy_score(y_test, rf_labels))
    
    #adaboost model
    ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate =.01).fit(x_train, y_train)
    #predict on test
    ada_pred = ada_md.predict_proba(x_test)[:,1]
    #changing likelihoods to labels w/ 10% cutoff
    ada_labels = np.where(ada_pred < .1, 0, 1)
    ada_recall.append(recall_score(y_test, ada_labels))
    ada_accuracy.append(accuracy_score(y_test, ada_labels))
    
    #gradient boost model
    gb_md= GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate =.01).fit(x_train, y_train)
    #predict on test
    gb_pred = gb_md.predict_proba(x_test)[:,1]
    #changing likelihoods to labels w/ 10% cutoff
    gb_labels = np.where(gb_pred < .1, 0, 1)
    gb_recall.append(recall_score(y_test, gb_labels))
    gb_accuracy.append(accuracy_score(y_test, gb_labels))

In [4]:
print('the avg recall for the random forest model:', np.mean(rf_recall))
print('the avg recall for the adaboost model:',np.mean(ada_recall))
print('the avg recall for the gradient boost model:',np.mean(gb_recall))

print('the avg accuracy for the random forest model:', np.mean(rf_accuracy))
print('the avg accuracy for the adaboost model:',np.mean(ada_accuracy))
print('the avg accuracy for the gradient boost model:',np.mean(gb_accuracy))

the avg recall for the random forest model: 0.8575
the avg recall for the adaboost model: 0.991875
the avg recall for the gradient boost model: 0.8067857142857144
the avg accuracy for the random forest model: 0.4508743169398908
the avg accuracy for the adaboost model: 0.15874316939890712
the avg accuracy for the gradient boost model: 0.5042622950819672


Though adaboost's recall is very high (.99), its accuracy score is very low (.16). Because of the accuracy, it cannot be considered the best model. That left me with the options of random forest (which is the middle option for both accuracy and recall) or gradient boost (highest accuracy and lowest recall but still close to random forest). Ultimately, the choice came down to higher recall or accuracy. Since we are dealing with disease, we don't want false negatives. Therefore, we'd rather the model with higher recall which is random forest. 

(c) (7 points) Assuming that the ideal model needs to have a minimum accuracy and recall equal to 80%. Does the best model from part (b) meet this requirement? If not, provide recommendations how you tweak the model to reach the minimum requirements. 

No, in fact, none of the models meet the requirement of accuaracy greater than or equal to 80%. In the case of the random forest model, we should try to tune our parameters like number of trees (n_estimators), how deep our trees are (max_depth), among other variables to improve the accuracy to 80%